In [1]:
!git clone https://github.com/mathilde-cros/Bachelor_Thesis_Alzheimers_HGNN.git

Cloning into 'Bachelor_Thesis_Alzheimers_HGNN'...
remote: Enumerating objects: 20191, done.
remote: Counting objects: 100% (2243/2243), done.
remote: Compressing objects: 100% (2123/2123), done.
remote: Total 20191 (delta 98), reused 2229 (delta 88), pack-reused 17948
Receiving objects: 100% (20191/20191), 5.16 GiB | 15.98 MiB/s, done.
Resolving deltas: 100% (3589/3589), done.
Updating files: 100% (4926/4926), done.


In [2]:
!pip install torch_geometric
!pip install torch_scatter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch_scatter: filename=torch_scatter-2.1.2-cp310-cp310-linux_x86_64.whl size=507268 sha256=50bf4d8559613ff7df29a93827148770cd85cdb6a1a9d085badd69d277287e99
  Stored in directory: /root/.cache/pip/wheels/92/f1/2b/3b46d54b134259f58c8363568569053248040859b1a145b3ce
Successfully built torch_scatter


In [3]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.0/289.0 kB 31.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.7 MB/s eta 0:00:00


In [4]:
## FUTURE WORK (NOT PART OF THESIS)

import math
import torch
import torch_geometric
import torch_scatter

import torch.nn as nn
import torch.nn.functional as F

from torch import Tensor
from torch.nn import Linear
from torch.nn import Parameter
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import softmax
from torch_scatter import scatter_add, scatter,  scatter_mean
from torch_geometric.typing import Adj, Size, OptTensor
from typing import Optional

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import wandb
import pdb
from torch_geometric.nn import global_mean_pool
from torch_geometric.utils import add_self_loops, degree

In [5]:
os.environ['WANDB_NOTEBOOK_NAME']="AllDeepSets.ipynb"

In [6]:
def glorot(tensor):
    if tensor is not None:
        stdv = math.sqrt(6.0 / (tensor.size(-2) + tensor.size(-1)))
        tensor.data.uniform_(-stdv, stdv)

In [7]:
class MLP(nn.Module):
    """ adapted from https://github.com/CUAI/CorrectAndSmooth/blob/master/gen_models.py """

    def __init__(self, in_channels, hidden_channels, out_channels, num_layers,
                 dropout=.5, Normalization='bn', InputNorm=False):
        super(MLP, self).__init__()
        self.lins = nn.ModuleList()
        self.normalizations = nn.ModuleList()
        self.InputNorm = InputNorm

        assert Normalization in ['bn', 'ln', 'None']
        if Normalization == 'bn':
            if num_layers == 1:
                # just linear layer i.e. logistic regression
                if InputNorm:
                    self.normalizations.append(nn.BatchNorm1d(in_channels))
                else:
                    self.normalizations.append(nn.Identity())
                self.lins.append(nn.Linear(in_channels, out_channels))
            else:
                if InputNorm:
                    self.normalizations.append(nn.BatchNorm1d(in_channels))
                else:
                    self.normalizations.append(nn.Identity())
                self.lins.append(nn.Linear(in_channels, hidden_channels))
                self.normalizations.append(nn.BatchNorm1d(hidden_channels))
                for _ in range(num_layers - 2):
                    self.lins.append(
                        nn.Linear(hidden_channels, hidden_channels))
                    self.normalizations.append(nn.BatchNorm1d(hidden_channels))
                self.lins.append(nn.Linear(hidden_channels, out_channels))
        elif Normalization == 'ln':
            if num_layers == 1:
                # just linear layer i.e. logistic regression
                if InputNorm:
                    self.normalizations.append(nn.LayerNorm(in_channels))
                else:
                    self.normalizations.append(nn.Identity())
                self.lins.append(nn.Linear(in_channels, out_channels))
            else:
                if InputNorm:
                    self.normalizations.append(nn.LayerNorm(in_channels))
                else:
                    self.normalizations.append(nn.Identity())
                self.lins.append(nn.Linear(in_channels, hidden_channels))
                self.normalizations.append(nn.LayerNorm(hidden_channels))
                for _ in range(num_layers - 2):
                    self.lins.append(
                        nn.Linear(hidden_channels, hidden_channels))
                    self.normalizations.append(nn.LayerNorm(hidden_channels))
                self.lins.append(nn.Linear(hidden_channels, out_channels))
        else:
            if num_layers == 1:
                # just linear layer i.e. logistic regression
                self.normalizations.append(nn.Identity())
                self.lins.append(nn.Linear(in_channels, out_channels))
            else:
                self.normalizations.append(nn.Identity())
                self.lins.append(nn.Linear(in_channels, hidden_channels))
                self.normalizations.append(nn.Identity())
                for _ in range(num_layers - 2):
                    self.lins.append(
                        nn.Linear(hidden_channels, hidden_channels))
                    self.normalizations.append(nn.Identity())
                self.lins.append(nn.Linear(hidden_channels, out_channels))

        self.dropout = dropout

    def reset_parameters(self):
        for lin in self.lins:
            lin.reset_parameters()
        for normalization in self.normalizations:
            if normalization.__class__.__name__ != 'Identity':
                normalization.reset_parameters()

    def forward(self, x):
        x = self.normalizations[0](x)
        for i, lin in enumerate(self.lins[:-1]):
            x = lin(x)
            x = F.relu(x, inplace=True)
            x = self.normalizations[i+1](x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.lins[-1](x)
        return x


class PMA(MessagePassing):
    """
        PMA part:
        Note that in original PMA, we need to compute the inner product of the seed and neighbor nodes.
        i.e. e_ij = a(Wh_i,Wh_j), where a should be the inner product, h_i is the seed and h_j are neightbor nodes.
        In GAT, a(x,y) = a^T[x||y]. We use the same logic.
    """
    _alpha: OptTensor

    def __init__(self, in_channels, hid_dim,
                 out_channels, num_layers, heads=1, concat=True,
                 negative_slope=0.2, dropout=0.0, bias=False, **kwargs):
        #         kwargs.setdefault('aggr', 'add')
        super(PMA, self).__init__(node_dim=0, **kwargs)

        self.in_channels = in_channels
        self.hidden = hid_dim // heads
        self.out_channels = out_channels
        self.heads = heads
        self.concat = concat
        self.negative_slope = negative_slope
        self.dropout = 0.
        self.aggr = 'add'
#         self.input_seed = input_seed

#         This is the encoder part. Where we use 1 layer NN (Theta*x_i in the GATConv description)
#         Now, no seed as input. Directly learn the importance weights alpha_ij.
#         self.lin_O = Linear(heads*self.hidden, self.hidden) # For heads combining
        # For neighbor nodes (source side, key)
        self.lin_K = Linear(in_channels, self.heads*self.hidden)
        # For neighbor nodes (source side, value)
        self.lin_V = Linear(in_channels, self.heads*self.hidden)
        self.att_r = Parameter(torch.Tensor(
            1, heads, self.hidden))  # Seed vector
        self.rFF = MLP(in_channels=self.heads*self.hidden,
                       hidden_channels=self.heads*self.hidden,
                       out_channels=out_channels,
                       num_layers=num_layers,
                       dropout=.0, Normalization='None',)
        self.ln0 = nn.LayerNorm(self.heads*self.hidden)
        self.ln1 = nn.LayerNorm(self.heads*self.hidden)
#         if bias and concat:
#             self.bias = Parameter(torch.Tensor(heads * out_channels))
#         elif bias and not concat:
#             self.bias = Parameter(torch.Tensor(out_channels))
#         else:

#         Always no bias! (For now)
        self.register_parameter('bias', None)

        self._alpha = None

        self.reset_parameters()

    def reset_parameters(self):
        #         glorot(self.lin_l.weight)
        glorot(self.lin_K.weight)
        glorot(self.lin_V.weight)
        self.rFF.reset_parameters()
        self.ln0.reset_parameters()
        self.ln1.reset_parameters()
#         glorot(self.att_l)
        nn.init.xavier_uniform_(self.att_r)
#         zeros(self.bias)

    def forward(self, x, edge_index: Adj,
                size: Size = None, return_attention_weights=None):
        # type: (Union[Tensor, OptPairTensor], Tensor, Size, NoneType) -> Tensor  # noqa
        # type: (Union[Tensor, OptPairTensor], SparseTensor, Size, NoneType) -> Tensor  # noqa
        # type: (Union[Tensor, OptPairTensor], Tensor, Size, bool) -> Tuple[Tensor, Tuple[Tensor, Tensor]]  # noqa
        # type: (Union[Tensor, OptPairTensor], SparseTensor, Size, bool) -> Tuple[Tensor, SparseTensor]  # noqa
        """
        Args:
            return_attention_weights (bool, optional): If set to :obj:`True`,
                will additionally return the tuple
                :obj:`(edge_index, attention_weights)`, holding the computed
                attention weights for each edge. (default: :obj:`None`)
        """
        H, C = self.heads, self.hidden

        x_l: OptTensor = None
        x_r: OptTensor = None
        alpha_l: OptTensor = None
        alpha_r: OptTensor = None
        if isinstance(x, Tensor):
            assert x.dim() == 2, 'Static graphs not supported in `GATConv`.'
            x_K = self.lin_K(x).view(-1, H, C)
            x_V = self.lin_V(x).view(-1, H, C)
            alpha_r = (x_K * self.att_r).sum(dim=-1)

        out = self.propagate(edge_index, x=x_V,
                             alpha=alpha_r, aggr=self.aggr)

        alpha = self._alpha
        self._alpha = None

#         Note that in the original code of GMT paper, they do not use additional W^O to combine heads.
#         This is because O = softmax(QK^T)V and V = V_in*W^V. So W^O can be effectively taken care by W^V!!!
        out += self.att_r  # This is Seed + Multihead
        # concat heads then LayerNorm. Z (rhs of Eq(7)) in GMT paper.
        out = self.ln0(out.view(-1, self.heads * self.hidden))
        # rFF and skip connection. Lhs of eq(7) in GMT paper.
        out = self.ln1(out+F.relu(self.rFF(out)))

        if isinstance(return_attention_weights, bool):
            assert alpha is not None
            if isinstance(edge_index, Tensor):
                return out, (edge_index, alpha)
            elif isinstance(edge_index, SparseTensor):
                return out, edge_index.set_value(alpha, layout='coo')
        else:
            return out

    def message(self, x_j, alpha_j,
                index, ptr,
                size_j):
        #         ipdb.set_trace()
        alpha = alpha_j
        alpha = F.leaky_relu(alpha, self.negative_slope)
        alpha = softmax(alpha, index, ptr, index.max()+1)
        self._alpha = alpha
        alpha = F.dropout(alpha, p=self.dropout, training=self.training)
        return x_j * alpha.unsqueeze(-1)

    def aggregate(self, inputs, index,
                  dim_size=None):
        r"""Aggregates messages from neighbors as
        :math:`\square_{j \in \mathcal{N}(i)}`.

        Takes in the output of message computation as first argument and any
        argument which was initially passed to :meth:`propagate`.

        By default, this function will delegate its call to scatter functions
        that support "add", "mean" and "max" operations as specified in
        :meth:`__init__` by the :obj:`aggr` argument.
        """
#         ipdb.set_trace()
        if self.aggr is None:
            raise ValueError("aggr was not passed!")
        return scatter(inputs, index, dim=self.node_dim, reduce=self.aggr)

    def __repr__(self):
        return '{}({}, {}, heads={})'.format(self.__class__.__name__,
                                             self.in_channels,
                                             self.out_channels, self.heads)


class HalfNLHconv(MessagePassing):
    def __init__(self,
                 in_dim,
                 hid_dim,
                 out_dim,
                 num_layers,
                 dropout,
                 Normalization='bn',
                 InputNorm=False,
                 heads=1,
                 attention=True
                 ):
        super(HalfNLHconv, self).__init__()

        self.attention = attention
        self.dropout = dropout

        if self.attention:
            self.prop = PMA(in_dim, hid_dim, out_dim, num_layers, heads=heads)
        else:
            if num_layers > 0:
                self.f_enc = MLP(in_dim, hid_dim, hid_dim, num_layers, dropout, Normalization, InputNorm)
                self.f_dec = MLP(hid_dim, hid_dim, out_dim, num_layers, dropout, Normalization, InputNorm)
            else:
                self.f_enc = nn.Identity()
                self.f_dec = nn.Identity()
#         self.bn = nn.BatchNorm1d(dec_hid_dim)
#         self.dropout = dropout
#         self.Prop = S2SProp()

    def reset_parameters(self):

        if self.attention:
            self.prop.reset_parameters()
        else:
            if self.f_enc.__class__.__name__ != 'Identity':
                self.f_enc.reset_parameters()
            if self.f_dec.__class__.__name__ != 'Identity':
                self.f_dec.reset_parameters()
#         self.bn.reset_parameters()

    def forward(self, x, edge_index, size):
        """
        input -> MLP -> Prop
        """

        if self.attention:
            x = self.prop(x, edge_index)
        else:
            x = F.relu(self.f_enc(x))
            x = F.dropout(x, p=self.dropout, training=self.training)
            x = self.propagate(edge_index, x=x, size=size)
            x = F.relu(self.f_dec(x))

        return x

    def message(self, x_j):
        out = x_j
        return out

    def aggregate(self, inputs, index,
                  dim_size=None):
        r"""Aggregates messages from neighbors as
        :math:`\square_{j \in \mathcal{N}(i)}`.

        Takes in the output of message computation as first argument and any
        argument which was initially passed to :meth:`propagate`.

        By default, this function will delegate its call to scatter functions
        that support "add", "mean" and "max" operations as specified in
        :meth:`__init__` by the :obj:`aggr` argument.
        """
#         ipdb.set_trace()
        if self.aggr is None:
            raise ValueError("aggr was not passed!")
        return scatter(inputs, index, dim=self.node_dim, reduce=self.aggr)

In [8]:
class SetGNN(nn.Module):
    def __init__(self, num_features,Classifier_hidden, num_classes,
                 Classifier_num_layers, MLP_hidden, MLP_num_layers,
                 All_num_layers, dropout, aggregate, normalization,
                 deepset_input_norm, GPR, use_PMA, heads, LearnMask, norm=None):
        super(SetGNN, self).__init__()
        """
        args should contain the following:
        V_in_dim, V_enc_hid_dim, V_dec_hid_dim, V_out_dim, V_enc_num_layers, V_dec_num_layers
        E_in_dim, E_enc_hid_dim, E_dec_hid_dim, E_out_dim, E_enc_num_layers, E_dec_num_layers
        All_num_layers,dropout
        !!! V_in_dim should be the dimension of node features
        !!! E_out_dim should be the number of classes (for classification)
        """

#         Now set all dropout the same, but can be different
        self.All_num_layers = All_num_layers
        self.dropout = dropout
        self.aggr = aggregate
        self.NormLayer = normalization
        self.InputNorm = deepset_input_norm
        self.GPR = GPR
        self.LearnMask =LearnMask
#         Now define V2EConvs[i], V2EConvs[i] for ith layers
#         Currently we assume there's no hyperedge features, which means V_out_dim = E_in_dim
#         If there's hyperedge features, concat with Vpart decoder output features [V_feat||E_feat]
        self.V2EConvs = nn.ModuleList()
        self.E2VConvs = nn.ModuleList()
        self.bnV2Es = nn.ModuleList()
        self.bnE2Vs = nn.ModuleList()

        if self.LearnMask:
            self.Importance = Parameter(torch.ones(norm.size()))

        if self.All_num_layers == 0:
            self.classifier = MLP(in_channels=num_features,
                                  hidden_channels=Classifier_hidden,
                                  out_channels=num_classes,
                                  num_layers=Classifier_num_layers,
                                  dropout=self.dropout,
                                  Normalization=self.NormLayer,
                                  InputNorm=False)
        else:
            self.V2EConvs.append(HalfNLHconv(in_dim=num_features,
                                             hid_dim=MLP_hidden,
                                             out_dim=MLP_hidden,
                                             num_layers=MLP_num_layers,
                                             dropout=self.dropout,
                                             Normalization=self.NormLayer,
                                             InputNorm=self.InputNorm,
                                             heads=heads,
                                             attention=use_PMA))
            self.bnV2Es.append(nn.BatchNorm1d(MLP_hidden))
            self.E2VConvs.append(HalfNLHconv(in_dim=MLP_hidden,
                                             hid_dim=MLP_hidden,
                                             out_dim=MLP_hidden,
                                             num_layers=MLP_num_layers,
                                             dropout=self.dropout,
                                             Normalization=self.NormLayer,
                                             InputNorm=self.InputNorm,
                                             heads=heads,
                                             attention=use_PMA))
            self.bnE2Vs.append(nn.BatchNorm1d(MLP_hidden))
            for _ in range(self.All_num_layers-1):
                self.V2EConvs.append(HalfNLHconv(in_dim=MLP_hidden,
                                                 hid_dim=MLP_hidden,
                                                 out_dim=MLP_hidden,
                                                 num_layers=MLP_num_layers,
                                                 dropout=self.dropout,
                                                 Normalization=self.NormLayer,
                                                 InputNorm=self.InputNorm,
                                                 heads=heads,
                                                 attention=use_PMA))
                self.bnV2Es.append(nn.BatchNorm1d(MLP_hidden))
                self.E2VConvs.append(HalfNLHconv(in_dim=MLP_hidden,
                                                 hid_dim=MLP_hidden,
                                                 out_dim=MLP_hidden,
                                                 num_layers=MLP_num_layers,
                                                 dropout=self.dropout,
                                                 Normalization=self.NormLayer,
                                                 InputNorm=self.InputNorm,
                                                 heads=heads,
                                                 attention=use_PMA))
                self.bnE2Vs.append(nn.BatchNorm1d(MLP_hidden))
            if self.GPR:
                self.MLP = MLP(in_channels=num_features,
                               hidden_channels=MLP_hidden,
                               out_channels=MLP_hidden,
                               num_layers=MLP_num_layers,
                               dropout=self.dropout,
                               Normalization=self.NormLayer,
                               InputNorm=False)
                self.GPRweights = Linear(self.All_num_layers+1, 1, bias=False)
                self.classifier = MLP(in_channels=MLP_hidden,
                                      hidden_channels=Classifier_hidden,
                                      out_channels=num_classes,
                                      num_layers=Classifier_num_layers,
                                      dropout=self.dropout,
                                      Normalization=self.NormLayer,
                                      InputNorm=False)
            else:
                self.classifier = MLP(in_channels=MLP_hidden,
                                      hidden_channels=Classifier_hidden,
                                      out_channels=num_classes,
                                      num_layers=Classifier_num_layers,
                                      dropout=self.dropout,
                                      Normalization=self.NormLayer,
                                      InputNorm=False)


#         Now we simply use V_enc_hid=V_dec_hid=E_enc_hid=E_dec_hid
#         However, in general this can be arbitrary.


    def reset_parameters(self):
        for layer in self.V2EConvs:
            layer.reset_parameters()
        for layer in self.E2VConvs:
            layer.reset_parameters()
        for layer in self.bnV2Es:
            layer.reset_parameters()
        for layer in self.bnE2Vs:
            layer.reset_parameters()
        self.classifier.reset_parameters()
        if self.GPR:
            self.MLP.reset_parameters()
            self.GPRweights.reset_parameters()
        if self.LearnMask:
            nn.init.ones_(self.Importance)

    def forward(self, x, edge_index, batch):
        """
        The data should contain the follows
        data.x: node features
        data.edge_index: edge list (of size (2,|E|)) where data.edge_index[0] contains nodes and data.edge_index[1] contains hyperedges
        !!! Note that self loop should be assigned to a new (hyper)edge id!!!
        !!! Also note that the (hyper)edge id should start at 0 (akin to node id)
        data.norm: The weight for edges in bipartite graphs, correspond to data.edge_index
        !!! Note that we output final node representation. Loss should be defined outside.
        """
#             The data should contain the follows
#             data.x: node features
#             data.V2Eedge_index:  edge list (of size (2,|E|)) where
#             data.V2Eedge_index[0] contains nodes and data.V2Eedge_index[1] contains hyperedges

        # x, edge_index, norm = data.x, data.edge_index, data.norm

        if self.LearnMask:
            norm = self.Importance
        cidx = edge_index[1].min()
        edge_index[1] -= cidx  # make sure we do not waste memory
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))
        reversed_edge_index = torch.stack(
            [edge_index[1], edge_index[0]], dim=0)
        if self.GPR:
            xs = []
            xs.append(F.relu(self.MLP(x)))
            for i, _ in enumerate(self.V2EConvs):
                x = F.relu(self.V2EConvs[i](x, edge_index, self.aggr))
                x = F.dropout(x, p=self.dropout, training=self.training)
                x = self.E2VConvs[i](x, reversed_edge_index,  self.aggr)
                x = F.relu(x)
                xs.append(x)
                x = F.dropout(x, p=self.dropout, training=self.training)
            x = torch.stack(xs, dim=-1)
            x = self.GPRweights(x).squeeze()
            x = global_mean_pool(x, batch)
            x = self.classifier(x)
        else:
            num_nodes = x.shape[0]
            num_edges = edge_index[1].max()+1
            x = F.dropout(x, p=0.2, training=self.training) # Input dropout
            for i, _ in enumerate(self.V2EConvs):
                x = F.relu(self.V2EConvs[i](x, edge_index, size=(num_nodes, num_edges)))

                x = F.dropout(x, p=self.dropout, training=self.training)

                x = F.relu(self.E2VConvs[i](
                    x, reversed_edge_index, size=(num_edges, num_nodes)))

                x = F.dropout(x, p=self.dropout, training=self.training)

            x = global_mean_pool(x, batch)
            x = self.classifier(x)

        return x

In [9]:
!pip install networkx

In [10]:
!pip install nilearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 62.0 MB/s eta 0:00:00


In [11]:
from torch_geometric.data import InMemoryDataset
from torch_geometric.utils import from_networkx
import networkx as nx
from networkx.convert_matrix import from_numpy_array
import pickle as pkl
from nilearn import datasets

In [12]:
# The corr matrices to build the hypergraphs

# Creating a dictionary of lists of paths to the correlation matrices for each method. Each list in the dictionary represents a different method.
## This is saved as corr_matrix_paths in functions.py
methods = ['pearson', 'spearman', 'kendall', 'partial']
full_corr_path_lists = {}
for method in methods:
    method_dir = f'/content/Bachelor_Thesis_Alzheimers_HGNN/ADNI_full/corr_matrices/corr_matrix_{method}'
    full_corr_path_lists[method] = []
    for file in os.listdir(method_dir):
        full_corr_path_lists[method].append(file)

# Generating the diagnostic label from the diagnostic_label.csv file
diagnostic_label = np.loadtxt('/content/Bachelor_Thesis_Alzheimers_HGNN/ADNI_full/diagnostic_label.csv', dtype=str, delimiter=',')

# Combining the 'EMCI', 'LMCI' and 'MCI' diagnostics into a single 'MCI' label for simplicity, then one-hot encoding the diagnostics
for patient in range(len(diagnostic_label)):
    if diagnostic_label[patient] == 'CN':
        diagnostic_label[patient] = 0
    elif diagnostic_label[patient] == 'SMC':
        diagnostic_label[patient] = 1
    elif diagnostic_label[patient] == 'EMCI' or diagnostic_label[patient] == 'LMCI' or diagnostic_label[patient] == 'MCI':
        diagnostic_label[patient] = 2
    elif diagnostic_label[patient] == 'AD':
        diagnostic_label[patient] = 3
    else:
        print('Error: Diagnostic label not recognised')
        break

In [13]:
# Useful for loading the hg_dicts
def load_hg_dict(root):
    dict_list = []
    for filename in os.listdir(root):
        path = os.path.join(root, filename)
        hg_dict = pkl.load(open(path, 'rb'))
        dict_list.append(hg_dict)
    return dict_list

# Creating a dictionary of lists of paths to the correlation matrices for each method. Each list in the dictionary represents a different method.
def corr_matrix_paths():
    methods = ['pearson', 'spearman', 'kendall', 'partial']
    full_corr_path_lists = {}
    for method in methods:
        method_dir = f'/content/Bachelor_Thesis_Alzheimers_HGNN/ADNI_full/corr_matrices/corr_matrix_{method}/'
        full_corr_path_lists[method] = []
        for file in os.listdir(method_dir):
            full_corr_path_lists[method].append(file)
    return full_corr_path_lists

# Combining the 'EMCI', 'LMCI' and 'MCI' diagnostics into a single 'MCI' label for simplicity, then one-hot encoding the diagnostics
def combine_diag_labels(diagnostic_label):
    for patient in range(len(diagnostic_label)):
        if diagnostic_label[patient] == 'CN':
            diagnostic_label[patient] = 0
        elif diagnostic_label[patient] == 'SMC':
            diagnostic_label[patient] = 1
        elif diagnostic_label[patient] == 'EMCI' or diagnostic_label[patient] == 'LMCI' or diagnostic_label[patient] == 'MCI':
            diagnostic_label[patient] = 2
        elif diagnostic_label[patient] == 'AD':
            diagnostic_label[patient] = 3
        else:
            print('Error: Diagnostic label not recognised')
            break
    return diagnostic_label

# Generating the brain atlas
def gen_atlas_labels():
    atlas = datasets.fetch_atlas_aal()
    atlas_filename = atlas.maps
    atlas_labels = atlas.labels
    n_ROIs = len(atlas_labels)
    return atlas, atlas_filename, atlas_labels, n_ROIs

In [14]:
!pip install hypernetx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 578.5/578.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 46.9 MB/s eta 0:00:00
  Attempting uninstall: decorator
    Found existing installation: decorator 4.4.2
    Uninstalling decorator-4.4.2:
      Successfully uninstalled decorator-4.4.2
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
  Attempting uninstall: pandas
    Found existing installation: pandas 2.0.3
    Uninstalling pandas-2.0.3:
      Successfully uninstalled pandas-2.0.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.

In [15]:
import hypernetx as hnx
from torch_geometric.data import InMemoryDataset
from torch_geometric.data import Data

ImportError: cannot import name '_fit_context' from 'sklearn.base' (/usr/local/lib/python3.10/dist-packages/sklearn/base.py)

In [ ]:
# Defining a function to display some statistics and features about the dataset.
def dataset_features_and_stats(dataset):
    print()
    print(f'Dataset: {dataset}:')
    print('====================')
    print(f'Number of graphs: {len(dataset)}')
    print(f'Weighted: {dataset.weight}')
    print(f'Threshold: {dataset.threshold}')
    print(f'Correlation Method: {dataset.method}')
    print(f'Number of features: {dataset.num_features}')
    print(f'Number of classes: {len(np.unique(diagnostic_label))}')

    # Getting the first graph object in the dataset.
    data = dataset[0]

    print()
    print(data)
    print('=============================================================')

    # Some statistics about the first graph.
    print(f'Number of nodes: {data.num_nodes}')
    print(f'Number of edges: {data.num_edges}')
    print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
    print(f'Has isolated nodes: {data.has_isolated_nodes()}')
    print(f'Has self-loops: {data.has_self_loops()}')
    print(f'Is undirected: {data.is_undirected()}')

    # Creating a NetworkX graph manually from the sample graph data
    sample_networkx_graph = nx.Graph()

    # Adding nodes with their attributes
    for i, feature_vector in enumerate(data.x):
        sample_networkx_graph.add_node(i, degree=feature_vector[0], betweenness_centrality=feature_vector[1], local_efficiency=feature_vector[2], cluster_coefficient=feature_vector[3], ratio_local_global=feature_vector[4])

    # Adding edges with no weights
    edge_index = data.edge_index
    for j in range(edge_index.shape[1]):
        node1 = edge_index[0][j].item()
        node2 = edge_index[1][j].item()
        sample_networkx_graph.add_edge(node1, node2, weight=1)

    # Plotting the sample graph
    plt.figure(figsize=(10, 6))
    sample_networkx_graph.remove_edges_from(nx.selfloop_edges(sample_networkx_graph))
    pos = nx.spring_layout(sample_networkx_graph, k=0.5)
    nx.draw(sample_networkx_graph, pos, with_labels=True)
    plt.title('Graph Visualization for patient 0')
    plt.show()

    print(data.y.detach().long())

In [ ]:
# Defining functions to simplify the code in the class Raw_to_Graph.

# To convert a dictionnary into a numpy array
def dict_to_array(dict):
    array = np.array(list(dict.values()))
    return array

# To normalize an array
def normalize_array(array):
    norm_array = (array - np.mean(array)) / np.std(array)
    return norm_array

# Defining a class to preprocess raw data into a format suitable for training Graph Neural Networks (GNNs).
## With the possibility of assigning weight to edges, adding the age feature, sex feature, and matrixe profiling.

class Raw_to_Hypergraph(InMemoryDataset):
    def __init__(self, root, hg_data_path, method, weight, threshold, age=False, sex=False, transform=None, pre_transform=None):
        self.method = method
        self.weight = weight
        self.threshold = threshold
        self.age = age
        self.sex = sex
        self.hg_data_path = hg_data_path
        super().__init__(root, transform, pre_transform)
        self.data, self.slices = torch.load(self.processed_paths[0])

    @property
    def processed_file_names(self):
        return ['data.pt']

    # This function is used to process the raw data into a format suitable for GNNs, by constructing graphs out of the connectivity matrices.
    def process(self):
        # Loading the prebuilt hypergraphs and the correlation matrices
        hg_dict_list = load_hg_dict(self.hg_data_path)
        full_corr_path_lists = corr_matrix_paths()
        corr_matrix_list = full_corr_path_lists['pearson']

        # Generating the diagnostic file from the diagnostic_label.csv file
        diagnostic_label = np.loadtxt('/content/Bachelor_Thesis_Alzheimers_HGNN/ADNI_full/diagnostic_label.csv', dtype=str, delimiter=',')
        # Combining the 'EMCI', 'LMCI' and 'MCI' diagnostics into a single 'MCI' label for simplicity, then one-hot encoding the diagnostics
        diagnostic_label = combine_diag_labels(diagnostic_label)

        graphs=[]
        for patient_idx, patient_hg in enumerate(hg_dict_list):
            # Create a NetworkX graph from the hypergraph matrix
            hypergraph = hnx.Hypergraph(patient_hg)
            print(type(patient_hg))
            print(patient_hg)
            print(len(patient_hg.keys()))

            # Adding the matrix profiling features to the feature array
            patient_matrix = corr_matrix_list[patient_idx]
            path = f'/content/Bachelor_Thesis_Alzheimers_HGNN/ADNI_full/matrix_profiles/matrix_profile_pearson/{patient_matrix}'
            if patient_matrix.endswith('.DS_Store'):
                continue  # Skip hidden system files like .DS_Store
            with open(path, "rb") as fl:
                patient_dict = pkl.load(fl)
            # combine dimensions
            features = np.array(patient_dict['mp']).reshape(len(patient_dict['mp']),-1)
            features = features.astype(np.float32)

            # # Compute the degree of each node
            # degree_dict = {}
            # for node in hypergraph.nodes():s
            #     degree_dict[node] = hypergraph.degree(node)

            # # Convert the degree to NumPy array then normalize it
            # degree_array = dict_to_array(degree_dict)
            # degree_array_norm = normalize_array(degree_array)
            # degree_array_norm = degree_array_norm.reshape(-1, 1)

            path_corr = f'/content/Bachelor_Thesis_Alzheimers_HGNN/ADNI_full/corr_matrices/corr_matrix_pearson/{corr_matrix_list[patient_idx]}'
            corr_array = np.loadtxt(path_corr, delimiter=',')

            # # Concatenate the degree, betweenness centrality, local efficiency, and ratio of local to global efficiency arrays to form a single feature vector
            # x_array = np.concatenate((degree_array_norm, corr_array), axis=-1)
            # x_array = x_array.astype(np.float32)

            # Concatenate the matrix profiling features to the feature array
            x_array = np.concatenate((corr_array, features), axis=-1)
            x_array = x_array.astype(np.float32)
            x = torch.tensor(x_array, dtype=torch.float)

            # Create a Pytorch Geometric Data object
            edge_index0 = []
            edge_index1 = []
            i = 0
            for hyperedge, nodes in hypergraph.incidence_dict.items():
                edge_index0 = np.concatenate((edge_index0, nodes), axis=0)
                for j in range(len(nodes)):
                    edge_index1.append(i)
                i += 1
            edge_index = np.stack([[int(x) for x in edge_index0], edge_index1], axis=0)
            y = torch.tensor(float(diagnostic_label[patient_idx]))
            hg_data = Data(x=x, edge_index=torch.tensor(edge_index, dtype=torch.long), y=y)
            graphs.append(hg_data)

        data, slices = self.collate(graphs)
        torch.save((data, slices), self.processed_paths[0])

In [ ]:
# Building the graphs
threshold = '3neighbors'
weight = False
age = False
sex = False
matrixprofile = True
method = 'knn'
w_decay = 0
lr = 0.001

root = f'/content/Bachelor_Thesis_Alzheimers_HGNN/Raw_to_hypergraph/ADNI_T_{threshold}_M_{method}_W{weight}_A{age}_S{sex}_MP{matrixprofile}'
hg_data_path = f'/content/Bachelor_Thesis_Alzheimers_HGNN/Hypergraphs/{method}/thresh_{threshold}'
dataset = Raw_to_Hypergraph(root=root, hg_data_path=hg_data_path, method=method, weight=weight, threshold=threshold, age=age, sex=sex)
dataset_features_and_stats(dataset)

# -------------------------------------------------------

num_features = 572 #number of features in the input
num_classes = 4 #number of classes in the output
Classifier_hidden = 64 #hidden dim in the last classification layer
Classifier_num_layers = 2 #number of layers in the last classification layer
MLP_hidden = 64 #hidden states in all the used MLPs
MLP_num_layers = 2 #number of layers in all the used MLPs
All_num_layers = 2 #number of All*Sets layers in the model
dropout = 0.0 #dropout
aggregate = 'mean' #type of aggregation (might be ignored, not sure)
normalization = 'ln' #type og normalisation it can be ln, bn or none
deepset_input_norm = False #if you norm the input or not
GPR = False #when set to True, the classification is made from all the intermendiate representations concatenated
use_PMA = False #when True, the model is AllSetTransformer, when False the model is AllDeepSets
heads = 1 #number of heads in Transformer
LearnMask = False

# num_nodes = dataset.nodes
# x = dataset.x
edge_index = dataset[0].edge_index
#norm = torch.ones_like(edge_index[0]) #would keep this like that all the time. it means we don't normalise the input.

model = SetGNN(num_features, Classifier_hidden, num_classes,
              Classifier_num_layers, MLP_hidden, MLP_num_layers,
              All_num_layers, dropout, aggregate, normalization,
              deepset_input_norm, GPR, use_PMA, heads, LearnMask)
# out = model(x, edge_index, norm)
# print(out.shape)

Training the model

In [ ]:
# The function we are using to compute the accuracy of our model
def quick_accuracy(y_hat, y):
  """
  Args :
    y_hat : logits predicted by model [n, num_classes]
    y : ground trutch labels [n]
  returns :
    average accuracy
  """
  n = y.shape[0]
  y_hat = torch.argmax(y_hat, dim=-1)
  accuracy = (y_hat==y).sum().data.item()
  return accuracy/n

In [ ]:
# Epoch Training

def epochs_training(model, optimizer, criterion, train_loader, valid_loader, test_loader, testing, test_accuracy, train_losses, train_accuracies, valid_losses, valid_accuracies, max_valid_accuracy):

    model.train()
    train_loss = 0
    train_accuracy = 0
    for data in train_loader:
        target = data.y.clone().detach().long()
        print(target)
        print(type(target))
        print(target.size())
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.batch)
        print(out)
        print(type(out))
        print(out.size())
        print(target.size())
        loss = criterion(out, target)
        loss.backward()
        optimizer.step()
        train_loss += criterion(out, target)
        train_accuracy += quick_accuracy(out, target)

    train_losses.append(train_loss.detach().numpy()/len(train_loader))
    train_accuracies.append(train_accuracy/len(train_loader))

    model.eval()
    valid_loss = 0
    valid_accuracy = 0
    with torch.no_grad():
        for data in valid_loader:
            target = data.y.clone().detach().long()
            out = model(data.x, data.edge_index, data.batch)
            valid_loss += criterion(out, target)
            valid_accuracy += quick_accuracy(out, target)

        valid_losses.append(valid_loss.detach().numpy()/len(valid_loader))
        valid_accuracies.append(valid_accuracy/len(valid_loader))
        if valid_accuracies[-1] > max_valid_accuracy:
            max_valid_accuracy = valid_accuracies[-1]
            if testing:
                test_accuracy = 0
                for data in test_loader:
                    target = data.y.clone().detach().long()
                    out = model(data.x, data.edge_index, data.batch)
                    test_accuracy += quick_accuracy(out, target)
                test_accuracy = test_accuracy/len(test_loader)

    if testing:
        return train_losses, train_accuracies, valid_losses, valid_accuracies, max_valid_accuracy, test_accuracy
    else:
        return train_losses, train_accuracies, valid_losses, valid_accuracies, max_valid_accuracy

In [ ]:
stratify = True
# Training the model
def train(model, optimizer, criterion, w_decay, threshold, method, train_loader, valid_loader, parameters, test_loader=False, testing=False, n_epochs=100):
    test_loader = test_loader
    testing = testing
    n_epochs = n_epochs

    train_losses = []
    train_accuracies = []
    valid_losses = []
    valid_accuracies = []
    max_valid_accuracy = 0
    test_accuracy = 0

    # start a new wandb run to track this script
    run = wandb.init(
        # set the wandb project where this run will be logged
        project = "Alzheimers_GNN",
        # track hyperparameters and run metadata
        config = {
        "architecture": "AllDeepSets",
        "method": method,
        "strat + w loss": stratify,
        "weights": weight,
        "corr_m node feature": True,
        "weight_decay": w_decay,
        "threshold": threshold,
        "matrix profiling": True,
        "learning_rate": parameters[0],
        "hidden_channels": parameters[1],
        "num_layers": parameters[2],
        "dropout": parameters[3],
        "heads": parameters[4],
        "epochs": n_epochs},)

    for epoch in range(n_epochs):
        if testing:
            train_losses, train_accuracies, valid_losses, valid_accuracies, max_valid_accuracy, test_accuracy = epochs_training(model, optimizer, criterion, train_loader, valid_loader, test_loader, testing, test_accuracy, train_losses, train_accuracies, valid_losses, valid_accuracies, max_valid_accuracy)
            wandb.log({"Train Loss": train_losses[-1], "Train Accuracy": train_accuracies[-1], "Validation Loss": valid_losses[-1], "Validation Accuracy": valid_accuracies[-1], "Max Valid Accuracy": max_valid_accuracy, "Test Accuracy": test_accuracy})
        else:
            train_losses, train_accuracies, valid_losses, valid_accuracies, max_valid_accuracy = epochs_training(model, optimizer, criterion, train_loader, valid_loader, test_loader, testing, test_accuracy, train_losses, train_accuracies, valid_losses, valid_accuracies, max_valid_accuracy)
            wandb.log({"Train Loss": train_losses[-1], "Train Accuracy": train_accuracies[-1], "Validation Loss": valid_losses[-1], "Validation Accuracy": valid_accuracies[-1], "Max Valid Accuracy": max_valid_accuracy})
        print(f'Epoch {epoch+1}/{n_epochs}')
        print(f'Train Loss: {train_losses[-1]:.4f}, Validation Loss: {valid_losses[-1]:.4f}')
        print(f'Train Accuracy: {train_accuracies[-1]:.4f}, Validation Accuracy: {valid_accuracies[-1]:.4f}')
        print(f'Max Validation Accuracy: {max_valid_accuracy:.4f}')

    if testing:
        print('Test Accuracy:', test_accuracy)

    plt.figure(figsize=(12, 5))

    # Plot Losses
    plt.subplot(1, 2, 1)
    plt.plot(train_losses, label=f'Train Loss')
    plt.plot(valid_losses, label=f'Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    # Plot Accuracy
    plt.subplot(1, 2, 2)
    plt.plot(train_accuracies, label=f'Train Accuracy')
    plt.plot(valid_accuracies, label=f'Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # Save the plot
    lr = parameters[0]
    hidden_channels = parameters[1]
    num_layers = parameters[2]
    dropout = parameters[3]
    filename = f'HGConv_Models_MP/threshold_{threshold}/method_{method}/lr{lr}_hc{hidden_channels}_nl{num_layers}_d{dropout}_epochs{n_epochs}_wdecay{w_decay}_w{weight}.png'
    # plt.savefig(filename)
    if testing:
        plt.title(f'Test Accuracy: {test_accuracy}')
    plt.show()

    wandb.finish()

    if testing:
        return train_losses, train_accuracies, valid_losses, valid_accuracies, max_valid_accuracy, test_accuracy
    else:
        return train_losses, train_accuracies, valid_losses, valid_accuracies, max_valid_accuracy


In [ ]:
from torch_geometric.data import DataLoader
from sklearn.model_selection import train_test_split

# Creating the train, validation and test sets
def create_train_test_valid(dataset, stratify=True, batch_size=16):
    X = dataset
    y = dataset.data.y
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
    X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.125, random_state=42, stratify=y_train)
    nbr_classes = len(np.unique(y))

    print(f'Number of training graphs: {len(X_train)}')
    print(f'Number of validation graphs: {len(X_valid)}')
    print(f'Number of test graphs: {len(X_test)}')
    print(f'Number of classes: {nbr_classes}')

    train_loader = DataLoader(X_train, batch_size=batch_size, shuffle=True)
    valid_loader = DataLoader(X_valid, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(X_test, batch_size=batch_size, shuffle=False)

    if stratify:
        return train_loader, valid_loader, test_loader, nbr_classes, y_train
    else:
        return train_loader, valid_loader, test_loader, nbr_classes, None

In [ ]:
parameters = [lr, Classifier_hidden, Classifier_num_layers, dropout, heads]
optimizer = torch.optim.Adam(model.parameters(), lr=parameters[0], weight_decay=w_decay)
criterion = torch.nn.CrossEntropyLoss()

train_loader, valid_loader, test_loader, nbr_classes, y_train = create_train_test_valid(dataset, stratify)


In [ ]:
for data in train_loader:
  out = model(data.x, data.edge_index, data.batch)
  print(out.size())
  print(out)


In [ ]:
# Running the training
train_losses, train_accuracies, valid_losses, valid_accuracies, max_valid_accuracy = train(model, optimizer, criterion, w_decay, threshold, method, train_loader, valid_loader, parameters, n_epochs=500)

In [ ]:
from itertools import product

In [ ]:
## Perform a Grid Search

# Define hyperparameters
w_decay_options = [0, 0.1, 0.001]
lr_options = [0.001, 0.01]
classifier_hidden_options = [32, 64, 128]
dropout_options = [0, 0.1, 0.2]
heads_options = [2, 3, 4]

# Define static parameters
threshold = '3neighbors'
weight = False
age = False
sex = False
matrixprofile = True
method = 'knn'
num_features = 572
num_classes = 4
classifier_num_layers = 2
mlp_hidden = 64
mlp_num_layers = 2
all_num_layers = 2
aggregate = 'mean'
normalization = 'ln'
deepset_input_norm = False
gpr = False
use_pma = False
learn_mask = False

# Grid search
for w_decay, lr, classifier_hidden, dropout, heads in product(w_decay_options, lr_options, classifier_hidden_options, dropout_options, heads_options):
    root = f'/content/Bachelor_Thesis_Alzheimers_HGNN/Raw_to_hypergraph/ADNI_T_{threshold}_M_{method}_W{weight}_A{age}_S{sex}_MP{matrixprofile}'
    hg_data_path = f'/content/Bachelor_Thesis_Alzheimers_HGNN/Hypergraphs/{method}/thresh_{threshold}'

    dataset = Raw_to_Hypergraph(root=root, hg_data_path=hg_data_path, method=method, weight=weight, threshold=threshold, age=age, sex=sex)
    dataset_features_and_stats(dataset)

    edge_index = dataset[0].edge_index

    model = SetGNN(num_features, classifier_hidden, num_classes,
                   classifier_num_layers, mlp_hidden, mlp_num_layers,
                   all_num_layers, dropout, aggregate, normalization,
                   deepset_input_norm, gpr, use_pma, heads, learn_mask)

    parameters = [lr, Classifier_hidden, Classifier_num_layers, dropout, heads]
    optimizer = torch.optim.Adam(model.parameters(), lr=parameters[0], weight_decay=w_decay)
    criterion = torch.nn.CrossEntropyLoss()

    train_loader, valid_loader, test_loader, nbr_classes, y_train = create_train_test_valid(dataset, stratify)

    # Running the training
    train_losses, train_accuracies, valid_losses, valid_accuracies, max_valid_accuracy = train(model, optimizer, criterion, w_decay, threshold, method, train_loader, valid_loader, parameters, n_epochs=500)

    print(f"Evaluated: w_decay={w_decay}, lr={lr}, classifier_hidden={classifier_hidden}, dropout={dropout}, heads={heads}")
